In [2]:
import pandas as pd
import os
import glob
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [3]:
# Load the datasets

job_postings_df = pd.read_csv('../datasets/job_postings/computing_desc_job_posting.csv')  # Adjust filename as needed



print("\nJob postings dataset structure:")
print(job_postings_df.head())
print(job_postings_df.columns)


Job postings dataset structure:
      job_id        company_name                        title  \
0   11009123   PGAV Destinations            project architect   
1   69333422     Staffing Theory    product marketing manager   
2  133130219                 NaN            software engineer   
3  175485704                GOYT            software engineer   
4  266825034  Recruitment Design  software support specialist   

                                         description  max_salary pay_period  \
0  PGAV Destinations is seeking a self-motivated ...         NaN        NaN   
1  A leading pharmaceutical company committed to ...         NaN        NaN   
2  Education Bachelor's degree in software, math,...         NaN        NaN   
3  Job Description:GOYT is seeking a skilled and ...         NaN        NaN   
4  Are you driven by the thrill of solving proble...     65000.0     YEARLY   

                        location  company_id  views  med_salary  ...  \
0                   St Louis,

In [6]:
# Load the datasets

resume_df = pd.read_csv('combined_resume_data.csv')  # Adjust filename as needed



print("\nResume dataset structure:")
print(resume_df.head())
print(resume_df.columns)


Resume dataset structure:
                                            raw_text                    role  \
0  C:\Workspace\java\scrape_indeed\dba_part_1\1.h...  Database Administrator   
1  C:\Workspace\java\scrape_indeed\dba_part_1\10....  Database Administrator   
2  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   
3  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   
4  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   

                                          clean_text  \
0  database administrator database administrator ...   
1  database administrator sql microsoft powerpoin...   
2  oracle database administrator oracle database ...   
3  amazon redshift administrator and etl develope...   
4  scrum master oracle database administrator scr...   

                  source_file  
0  Database_Administrator.csv  
1  Database_Administrator.csv  
2  Database_Administrator.csv  
3  Database_Administrator.c

In [4]:
job_postings_df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,work_type,currency,compensation_type,normalized_salary,zip_code,fips,matched_category,standardized_salary,description_length,cleaned_description
0,11009123,PGAV Destinations,project architect,PGAV Destinations is seeking a self-motivated ...,NaN,NaN,"St Louis, MO",93456.0,2.0,NaN,...,FULL_TIME,NaN,NaN,NaN,63367.0,29183.0,software architect,0.0,253,pgav destination seeking highly creative indiv...
1,69333422,Staffing Theory,product marketing manager,A leading pharmaceutical company committed to ...,NaN,NaN,United States,27097933.0,2.0,NaN,...,FULL_TIME,NaN,NaN,NaN,NaN,NaN,product manager,0.0,277,leading pharmaceutical company committed devel...
2,133130219,NaN,software engineer,"Education Bachelor's degree in software, math,...",NaN,NaN,Los Angeles Metropolitan Area,NaN,1.0,NaN,...,FULL_TIME,NaN,NaN,NaN,NaN,NaN,software engineer,0.0,23,education bachelor degree software math scienc...
3,175485704,GOYT,software engineer,Job Description:GOYT is seeking a skilled and ...,NaN,NaN,"Denver, CO",76987056.0,273.0,NaN,...,PART_TIME,NaN,NaN,NaN,80202.0,8031.0,software engineer,0.0,223,job description goyt seeking skilled motivated...
4,266825034,Recruitment Design,software support specialist,Are you driven by the thrill of solving proble...,65000.0,YEARLY,"McLean, VA",99212509.0,NaN,NaN,...,FULL_TIME,USD,BASE_SALARY,62500.0,22101.0,51059.0,software engineer,65000.0,177,driven thrill solving problem offering unparal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12277,3906264116,Western Integrated Technologies,hydraulic systems engineer,Are you a dynamic and experienced Hydraulic Sy...,NaN,NaN,"Sacramento, CA",522987.0,4.0,NaN,...,FULL_TIME,NaN,NaN,NaN,95811.0,6067.0,systems engineer,0.0,557,dynamic experienced hydraulic system engineer ...
12278,3906264345,"TalentBurst, an Inc 5000 company",quality engineer,Position: Quality Engineer I/IILocation: Irvin...,50.0,HOURLY,"Irvine, CA",122451.0,2.0,NaN,...,CONTRACT,USD,BASE_SALARY,93600.0,92602.0,NaN,software quality assurance engineer,104000.0,448,position quality engineer irvine ca onsite dur...
12279,3906266217,The Dyrt,frontend/app developer,The Dyrt is the largest digital camping platfo...,NaN,NaN,United States,6404239.0,1.0,NaN,...,FULL_TIME,NaN,NaN,NaN,NaN,NaN,frontend engineer,0.0,1122,dyrt largest digital camping platform world ra...
12280,3906266272,"TalentBurst, an Inc 5000 company",quality engineer,Position: Quality Engineer I (Complaint Invest...,50.0,HOURLY,"Irvine, CA",122451.0,1.0,NaN,...,CONTRACT,USD,BASE_SALARY,83200.0,92602.0,NaN,software quality assurance engineer,104000.0,227,position quality engineer complaint investigat...


In [6]:
import pandas as pd
import os
import glob

# Define the directory where your job CSV files are located
directory_path = '../datasets/resumes'  # Replace with your actual directory path

# Create an empty list to store each DataFrame
all_dfs = []

# Option 1: Using glob to find all CSV files
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
for file_path in csv_files:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Optional: Add a column indicating the source file
    file_name = os.path.basename(file_path)
    df['source_file'] = file_name
    
    # Append to our list of DataFrames
    all_dfs.append(df)
    print(f"Loaded: {file_name}, Shape: {df.shape}")

# Option 2: If you have specific file names or patterns
# job_files = ['job1.csv', 'job2.csv', 'job3.csv']  # Replace with your file names
# for file_name in job_files:
#     file_path = os.path.join(directory_path, file_name)
#     df = pd.read_csv(file_path)
#     df['source_file'] = file_name
#     all_dfs.append(df)
#     print(f"Loaded: {file_name}, Shape: {df.shape}")

# Combine all DataFrames into one
combined_df = pd.concat(all_dfs, ignore_index=True)
print(f"Combined dataset shape: {combined_df.shape}")

# Check for and handle any column inconsistencies
print(f"Columns in combined dataset: {combined_df.columns.tolist()}")

# Save the combined dataset
combined_df.to_csv('combined_resume_data.csv', index=False)
print("Combined dataset saved as 'combined_resume_data.csv'")

Loaded: Database_Administrator.csv, Shape: (3069, 4)
Loaded: FrontEnd_Developer.csv, Shape: (3040, 4)
Loaded: IT_Project_Manager.csv, Shape: (3085, 4)
Loaded: IT_Security_Analyst.csv, Shape: (3032, 4)
Loaded: Java_Dev.csv, Shape: (2432, 4)
Loaded: Network_Administrator.csv, Shape: (2967, 4)
Loaded: Python_Developer.csv, Shape: (2883, 4)
Loaded: Software_Developer.csv, Shape: (2923, 4)
Loaded: Software_Developer1.csv, Shape: (293, 4)
Loaded: Software_Developer10.csv, Shape: (292, 4)
Loaded: Software_Developer2.csv, Shape: (293, 4)
Loaded: Software_Developer3.csv, Shape: (293, 4)
Loaded: Software_Developer4.csv, Shape: (292, 4)
Loaded: Software_Developer5.csv, Shape: (292, 4)
Loaded: Software_Developer7.csv, Shape: (292, 4)
Loaded: Software_Developer8.csv, Shape: (292, 4)
Loaded: Software_Developer9.csv, Shape: (292, 4)
Loaded: System_Administrator.csv, Shape: (3257, 4)
Loaded: Web_Developer.csv, Shape: (2932, 4)
Combined dataset shape: (32251, 4)
Columns in combined dataset: ['raw_text'

In [7]:
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:

# Step 2: Preprocess text function
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return ' '.join(lemmatized_words)

In [10]:
# Step 3: Process both datasets
def prepare_datasets(resumes_df, job_postings_df):
    # Process resumes
    print("Processing resume data...")
    if 'clean_text' not in resumes_df.columns:
        resumes_df['processed_text'] = resumes_df['raw_text'].apply(preprocess_text)
    else:
        resumes_df['processed_text'] = resumes_df['clean_text'].apply(preprocess_text)
    
    # Add role as a feature if available
    if 'role' in resumes_df.columns:
        resumes_df['processed_role'] = resumes_df['role'].apply(preprocess_text)
        resumes_df['combined_resume_text'] = resumes_df['processed_text'] + ' ' + resumes_df['processed_role']
    else:
        resumes_df['combined_resume_text'] = resumes_df['processed_text']
    
    # Process job postings
    print("Processing job posting data...")
    # Using cleaned_description if available, otherwise use description
    if 'cleaned_description' in job_postings_df.columns:
        job_postings_df['processed_description'] = job_postings_df['cleaned_description'].apply(preprocess_text)
    else:
        job_postings_df['processed_description'] = job_postings_df['description'].apply(preprocess_text)
    
    # Process title and skills if available
    if 'title' in job_postings_df.columns:
        job_postings_df['processed_title'] = job_postings_df['title'].apply(preprocess_text)
    
    if 'skills_desc' in job_postings_df.columns and job_postings_df['skills_desc'].notna().any():
        job_postings_df['processed_skills'] = job_postings_df['skills_desc'].apply(preprocess_text)
        job_postings_df['combined_job_text'] = (
            job_postings_df['processed_description'] + ' ' + 
            job_postings_df['processed_title'] + ' ' + 
            job_postings_df['processed_skills'].fillna('')
        )
    else:
        job_postings_df['combined_job_text'] = (
            job_postings_df['processed_description'] + ' ' + 
            job_postings_df['processed_title'].fillna('')
        )
    
    return resumes_df, job_postings_df


In [9]:
resume_df

,raw_text,role,clean_text,source_file
0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator,database administrator database administrator ...,Database_Administrator.csv
1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,Database Administrator,database administrator sql microsoft powerpoin...,Database_Administrator.csv
2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,oracle database administrator oracle database ...,Database_Administrator.csv
3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,amazon redshift administrator and etl develope...,Database_Administrator.csv
4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,scrum master oracle database administrator scr...,Database_Administrator.csv
...,...,...,...,...
32246,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,servicenow developer servicenow developer serv...,Web_Developer.csv
32247,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,android developer android developer route plan...,Web_Developer.csv
32248,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,ui developer ui web developer ui developer ui ...,Web_Developer.csv
32249,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,web developer javascript php front end wordpre...,Web_Developer.csv


In [11]:

# 3. Prepare both datasets
processed_resumes_df, processed_job_postings_df = prepare_datasets(resume_df, job_postings_df)

# 4. Save processed datasets
processed_resumes_df.to_csv('processed_resumes.csv', index=False)
processed_job_postings_df.to_csv('processed_job_postings.csv', index=False)

print("Data preparation complete!")

Processing resume data...
Processing job posting data...
Data preparation complete!


In [12]:
# Step 1: Build TF-IDF Vectors
def calculate_tfidf_similarity(resumes_df, job_postings_df):
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    
    # Combine all text documents to build the vocabulary
    all_documents = list(resumes_df['combined_resume_text']) + list(job_postings_df['combined_job_text'])
    tfidf_vectorizer.fit(all_documents)
    
    # Transform resume and job posting text to TF-IDF vectors
    resume_vectors = tfidf_vectorizer.transform(resumes_df['combined_resume_text'])
    job_vectors = tfidf_vectorizer.transform(job_postings_df['combined_job_text'])
    
    # Calculate similarity between each resume and each job posting
    similarity_matrix = cosine_similarity(resume_vectors, job_vectors)
    
    # Create a DataFrame to store similarity scores
    similarity_df = pd.DataFrame(
        similarity_matrix, 
        index=resumes_df.index,
        columns=job_postings_df['job_id']
    )
    
    return similarity_df, tfidf_vectorizer

In [13]:
# Call the function
tfidf_similarity_df, tfidf_vectorizer = calculate_tfidf_similarity(processed_resumes_df, processed_job_postings_df)


In [14]:
# Find top job matches for each resume
def get_top_matches(similarity_df, n=5):
    top_matches = {}
    
    for idx in similarity_df.index:
        # Sort jobs by similarity score for this resume
        resume_scores = similarity_df.loc[idx].sort_values(ascending=False)
        # Get top n job IDs and scores
        top_jobs = resume_scores.head(n)
        top_matches[idx] = top_jobs
    
    return top_matches

In [15]:

resume_top_matches = get_top_matches(tfidf_similarity_df)

In [ ]:
resume_top_matches

In [17]:
def interpret_resume_matches(resume_top_matches, resumes_df, job_postings_df, n=5):
    """
    Create a more interpretable format for the top matches of each resume.
    
    Parameters:
    -----------
    resume_top_matches : dict
        Dictionary of top job matches for each resume
    resumes_df : pandas.DataFrame
        DataFrame containing resume data
    job_postings_df : pandas.DataFrame
        DataFrame containing job posting data
    n : int
        Number of top matches to include (default=5)
    
    Returns:
    --------
    pandas.DataFrame
        A DataFrame with detailed match information
    """
    results = []
    
    for resume_idx, job_matches in resume_top_matches.items():
        # Get resume information
        if 'role' in resumes_df.columns:
            resume_role = resumes_df.loc[resume_idx, 'role']
        else:
            resume_role = "Unknown"
            
        # Get source file for identification
        resume_source = resumes_df.loc[resume_idx, 'source_file']
        
        # Process each top match
        for rank, (job_id, similarity_score) in enumerate(job_matches.items(), 1):
            # Find the job in the job_postings DataFrame
            job_row = job_postings_df[job_postings_df['job_id'] == job_id]
            
            if not job_row.empty:
                job_title = job_row['title'].values[0]
                job_company = job_row['company_name'].values[0] if 'company_name' in job_row.columns else "Unknown"
                
                # Optional: Get salary information if available
                if 'normalized_salary' in job_row.columns:
                    job_salary = job_row['normalized_salary'].values[0]
                else:
                    job_salary = None
                
                # Create a row for the results DataFrame
                result_row = {
                    'Resume_ID': resume_idx,
                    'Resume_Source': resume_source,
                    'Resume_Role': resume_role,
                    'Match_Rank': rank,
                    'Job_ID': job_id,
                    'Job_Title': job_title,
                    'Company': job_company,
                    'Similarity_Score': similarity_score,
                    'Salary': job_salary
                }
                
                results.append(result_row)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by resume and then by rank
    results_df = results_df.sort_values(['Resume_ID', 'Match_Rank'])
    
    return results_df

In [18]:
# Call the function
interpretable_matches = interpret_resume_matches(resume_top_matches, processed_resumes_df, processed_job_postings_df)

# Save to CSV for easy viewing
interpretable_matches.to_csv('resume_job_matches.csv', index=False)

# Display a sample of the results
print(interpretable_matches.head(10))


   Resume_ID               Resume_Source             Resume_Role  Match_Rank  \
0          0  Database_Administrator.csv  Database Administrator           1   
1          0  Database_Administrator.csv  Database Administrator           2   
2          0  Database_Administrator.csv  Database Administrator           3   
3          0  Database_Administrator.csv  Database Administrator           4   
4          0  Database_Administrator.csv  Database Administrator           5   
5          1  Database_Administrator.csv  Database Administrator           1   
6          1  Database_Administrator.csv  Database Administrator           2   
7          1  Database_Administrator.csv  Database Administrator           3   
8          1  Database_Administrator.csv  Database Administrator           4   
9          1  Database_Administrator.csv  Database Administrator           5   

       Job_ID                        Job_Title  \
0  3905830762                    data engineer   
1  3906082919      

In [19]:
interpretable_matches.columns

Index(['Resume_ID', 'Resume_Source', 'Resume_Role', 'Match_Rank', 'Job_ID',
       'Job_Title', 'Company', 'Similarity_Score', 'Salary'],
      dtype='object')

In [20]:
interpretable_matches = interpretable_matches.sort_values(by='Similarity_Score', ascending=False)


In [32]:
interpretable_matches

,Resume_ID,Resume_Source,Resume_Role,Match_Rank,Job_ID,Job_Title,Company,Similarity_Score,Salary
43145,8629,IT_Project_Manager.csv,IT Project Manager,1,3902367568,tech java h/f,LesJeudis,0.917260,NaN
161005,32201,Web_Developer.csv,Web Developer,1,3902367568,tech java h/f,LesJeudis,0.887117,NaN
109820,21964,Software_Developer.csv,Software Developer,1,3902367568,tech java h/f,LesJeudis,0.855936,NaN
125895,25179,Software_Developer5.csv,Software Developer,1,3902367568,tech java h/f,LesJeudis,0.855936,NaN
43146,8629,IT_Project_Manager.csv,IT Project Manager,2,3904570046,[finance] business analytics,will bank,0.843746,NaN
...,...,...,...,...,...,...,...,...,...
118436,23687,Software_Developer1.csv,NaN,2,3903805988,warehouse technician,Groundworks,0.000000,NaN
118439,23687,Software_Developer1.csv,NaN,5,3903805067,embedded software engineer,Mainspring Energy,0.000000,NaN
118438,23687,Software_Developer1.csv,NaN,4,3903804930,product manager - blockchain,Confidencial,0.000000,NaN
118437,23687,Software_Developer1.csv,NaN,3,3903804811,project accountant,"CHA Consulting, Inc.",0.000000,NaN


In [27]:
job_postings_df[job_postings_df['job_id'] == 3902367568	]['cleaned_description'].values[0]

'mission nous somme une esn agile un groupe international certifié top employer europe de la digitale nous mettons tout en œuvre pour que chacun de no athlètes du digital puisse se renouveler perpétuellement en vivant positivement son propre flow de no talent peut ainsi façonner son parcours de carrière selon s appétences entreprendre de manière pragmatique avec s client pour un monde à impact positif innover localement dans pay et harmoniser son investissement professionnel et son personnel rejoignez inetum live positive digital flow tous no postes sont ouverts aux personnes en situation de handicap inetum recherche de tech lead java pour encadrer une équipe de personnes dont deux junior en leur fournissant de conseils et de orientation technique dans ce contexte que vous devrez réaliser le mission suivantes concevoir et développer de application java de haute qualitétravailler en étroite collaboration avec le autres membres de pour assurer la qualité et la cohérence du codeparticiper

In [35]:
resume_df[resume_df['source_file'] == "IT_Project_Manager.csv"]['processed_text']

6109    project manager manager self employed php soft...
6110    program manager project manager business syste...
6111    project manager project specialist jr architec...
6112    program manager senior project manager pmo sen...
6113    senior vmo project manager consultant supply c...
                              ...                        
9189    program manager project manager pmo project ma...
9190    project coordinator manager senior finance bus...
9191    business analyst project manager project manag...
9192    project manager network system security admini...
9193    web developer financial analyst manager corpor...
Name: processed_text, Length: 3085, dtype: object

In [33]:
resume_df.columns

Index(['raw_text', 'role', 'clean_text', 'source_file', 'processed_text',
       'processed_role', 'combined_resume_text'],
      dtype='object')